<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Manipulated_Media_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Deepfake Detection and Multimodal Manipulated Media Analysis Pipeline for Defensive Forensic**



This end-to-end pipeline uses LangChain-based VLLM and Groq LLM integration
along with LangGraph multiagent orchestration to analyze audio, video, and image modalities.
Each modality has its own processing and analysis steps, and the results are aggregated
into a consolidated report

In [3]:
!pip install -q torch numpy opencv-python librosa pydantic mediapipe moviepy face_recognition scikit-image dtw-python scipy langchain langchain_community langchain_core langgraph nest_asyncio ipywidgets mcp_use langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.3 MB/s eta 0:00:00


In [4]:
import os
import torch
import cv2
import numpy as np
import librosa
import asyncio
import json
import re
import gc
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List, Any, Tuple, Optional

# Media processing libraries
import mediapipe as mp
from pydantic import BaseModel, Field
from moviepy.editor import VideoFileClip

# For interactive notebook usage
import nest_asyncio
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# For image quality metrics
from skimage.metrics import structural_similarity as ssim
from dtw import dtw
import face_recognition
from scipy.spatial.distance import cosine

# Integration for LLM inference using vLLM and Groq.
from langchain_community.llms import VLLM
from langchain_groq import ChatGroq
# MCP-Use integration supporting multi-server configuration.
from mcp_use import MCPAgent, MCPClient

# Graph orchestration for pipeline steps.
from langgraph.graph import StateGraph, END

# Apply nest_asyncio to allow running async code in notebooks.
nest_asyncio.apply()
torch.cuda.empty_cache()

In [5]:
DEBUG = True
def debug_print(msg: str):
    """Print debug messages when DEBUG is enabled."""
    if DEBUG:
        print(f"[DEBUG] {msg}")

In [7]:
COMMON_PARAMS = {
    "task": "generate",
    "max_model_len": 4096,
    "dtype": "half",
    "gpu_memory_utilization": 0.85,
    "cpu_offload_gb": 8,
    "enforce_eager": True,
    "trust_remote_code": True
}

def init_vllm_model(name: str, model_id: str, **overrides):
    params = {**COMMON_PARAMS, **overrides}
    debug_print(f"Initializing VLLM model '{name}' with id '{model_id}' and params: {params}")
    return {"name": name, "model_id": model_id, "params": params}

def init_groq_model(name: str, model_id: str):
    api_key = os.environ.get("GROQ_API_KEY", "your_groq_api_key")
    debug_print(f"Initializing Groq model '{name}' with id '{model_id}' using API key.")
    return {"name": name, "model_id": model_id, "api_key": api_key}

# Groq LLM wrapper. In production you might call Groq’s specialized endpoints.
class GroqLLMWrapper:
    def __init__(self, model_data):
        self.model_data = model_data

    def __call__(self, prompt: str) -> str:
        debug_print(f"GroqLLMWrapper processing prompt with {self.model_data['name']}")
        # Simulated response from Groq engine.
        if "audio" in self.model_data["name"]:
            return "Score: 0.82\nAnomalies: [\"unnatural voice transitions\", \"inconsistent background noise\"]"
        elif "video" in self.model_data["name"]:
            return "Score: 0.77\nAnomalies: [\"facial landmark inconsistencies\", \"unnatural eye movements\"]"
        elif "image" in self.model_data["name"]:
            return "Score: 0.68\nAnomalies: [\"lighting inconsistencies\", \"unusual facial proportions\"]"
        else:
            return "Score: 0.75\nAnomalies: [\"inconsistent narrative\", \"unusual phrasing patterns\"]"

# Simulated vLLM wrapper using the VLLM library.
class VLLMSimulated:
    def __init__(self, model_data):
        self.model_data = model_data

    def __call__(self, prompt: str) -> str:
        debug_print(f"VLLMSimulated processing with {self.model_data['name']}")
        if "wav2vec" in self.model_data["name"] or "whisper" in self.model_data["name"]:
            return "Score: 0.79\nAnomalies: [\"frequency anomalies\", \"unnatural pauses\"]"
        elif "video" in self.model_data["name"]:
            return "Score: 0.81\nAnomalies: [\"temporal inconsistencies\", \"blending artifacts\"]"
        elif "llava" in self.model_data["name"] or "clip" in self.model_data["name"]:
            return "Score: 0.73\nAnomalies: [\"compression artifacts\", \"unusual texture patterns\"]"
        else:
            return "Score: 0.78\nAnomalies: [\"stylistic inconsistencies\"]"

# Initialize modality-specific models using vLLM and Groq wrappers.
debug_print("Initializing modality-specific models...")
models = {
    "audio": {
        "vllm": [
            VLLMSimulated(init_vllm_model("wav2vec2", "facebook/wav2vec2-large-robust-ft-swbd-300h", tensor_parallel_size=1)),
            VLLMSimulated(init_vllm_model("whisper", "openai/whisper-large-v3", tensor_parallel_size=2))
        ],
        "groq": [
            GroqLLMWrapper(init_groq_model("groq_audio_model", "whisper-large-v3-turbo")),
            GroqLLMWrapper(init_groq_model("groq_llama_audio", "meta-llama/llama-4-audio-17b-16e-instruct"))
        ]
    },
    "video": {
        "vllm": [
            VLLMSimulated(init_vllm_model("llava_next_video", "llava-hf/LLaVA-NeXT-Video-7B-hf", tensor_parallel_size=2, max_tokens=1024)),
            VLLMSimulated(init_vllm_model("videomae", "MCG-NJU/videomae-large-static", tensor_parallel_size=2))
        ],
        "groq": [
            GroqLLMWrapper(init_groq_model("groq_video_scout", "meta-llama/llama-4-scout-17b-16e-instruct")),
            GroqLLMWrapper(init_groq_model("groq_video_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct"))
        ]
    },
    "image": {
        "vllm": [
            VLLMSimulated(init_vllm_model("llava_image", "llava-hf/llava-onevision-qwen2-7b-ov-hf", tensor_parallel_size=2)),
            VLLMSimulated(init_vllm_model("clip", "openai/clip-vit-large-patch14", tensor_parallel_size=1))
        ],
        "groq": [
            GroqLLMWrapper(init_groq_model("groq_image_scout", "meta-llama/llama-4-scout-17b-16e-instruct")),
            GroqLLMWrapper(init_groq_model("groq_image_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct"))
        ]
    },
    "text": {
        "vllm": [
            VLLMSimulated(init_vllm_model("llama3", "meta-llama/Llama-3-70b-hf", tensor_parallel_size=4))
        ],
        "groq": [
            GroqLLMWrapper(init_groq_model("groq_text_scout", "meta-llama/llama-4-scout-17b-16e-instruct")),
            GroqLLMWrapper(init_groq_model("groq_text_maverick", "meta-llama/llama-4-maverick-17b-128e-instruct"))
        ]
    }
}

[DEBUG] Initializing modality-specific models...
[DEBUG] Initializing VLLM model 'wav2vec2' with id 'facebook/wav2vec2-large-robust-ft-swbd-300h' and params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 1}
[DEBUG] Initializing VLLM model 'whisper' with id 'openai/whisper-large-v3' and params: {'task': 'generate', 'max_model_len': 4096, 'dtype': 'half', 'gpu_memory_utilization': 0.85, 'cpu_offload_gb': 8, 'enforce_eager': True, 'trust_remote_code': True, 'tensor_parallel_size': 2}
[DEBUG] Initializing Groq model 'groq_audio_model' with id 'whisper-large-v3-turbo' using API key.
[DEBUG] Initializing Groq model 'groq_llama_audio' with id 'meta-llama/llama-4-audio-17b-16e-instruct' using API key.
[DEBUG] Initializing VLLM model 'llava_next_video' with id 'llava-hf/LLaVA-NeXT-Video-7B-hf' and params: {'task': 'generate', 'max_model_len': 4096, 'dtype

In [8]:
class DeepfakeAnalysisResult(BaseModel):
    score: float
    label: str
    anomalies: List[str] = Field(default_factory=list)
    artifacts: List[str] = Field(default_factory=list)
    confidence: float
    method: str
    timestamp: datetime = Field(default_factory=datetime.now)
    explanation: Optional[str] = None
    model_scores: Dict[str, float] = Field(default_factory=dict)

class Evidence(BaseModel):
    type: str
    description: str
    confidence: float
    method: str
    timestamp: Optional[float] = None
    location: Optional[Dict[str, int]] = None

class MultimodalAnalysisReport(BaseModel):
    case_id: str
    file_info: Dict[str, Any]
    video_analysis: Optional[DeepfakeAnalysisResult]
    audio_analysis: Optional[DeepfakeAnalysisResult]
    image_analysis: Optional[DeepfakeAnalysisResult]
    text_analysis: Optional[DeepfakeAnalysisResult]
    multimodal_score: float
    verdict: str
    evidence: List[Evidence]
    metadata: Dict[str, Any]
    recommendations: List[str] = Field(default_factory=list)
    confidence_matrix: Dict[str, Dict[str, float]] = Field(default_factory=dict)
    processing_time: float

In [9]:
def detect_face_forgery(image: np.ndarray) -> Tuple[float, List[str]]:
    """
    Additional check on facial regions to detect forgery artifacts.
    """
    score = 0.75
    anomalies = []
    avg_brightness = np.mean(image)
    if avg_brightness < 100:
        anomalies.append("Low brightness may indicate face forgery artifacts")
        score = 0.6
    return score, anomalies

def check_background_consistency(frames: List[np.ndarray]) -> Tuple[float, List[str]]:
    """
    Check for consistency in background color across video frames.
    """
    score = 0.8
    anomalies = []
    if len(frames) >= 2:
        bg_first = np.median(frames[0], axis=(0,1))
        bg_last = np.median(frames[-1], axis=(0,1))
        color_diff = np.linalg.norm(bg_first - bg_last)
        if color_diff > 30:
            anomalies.append("Inconsistent background detected across frames")
            score = 0.65
    return score, anomalies

def additional_audio_checks(audio_data: np.ndarray, sr: int) -> Tuple[float, List[str]]:
    """
    Additional audio checks for echoes and background noise.
    """
    score = 0.8
    anomalies = []
    variance = np.var(audio_data)
    if variance < 0.001:
        anomalies.append("Audio may suffer from echo or lack of clarity")
        score = 0.65
    if np.std(np.abs(audio_data)) > 0.5:
        anomalies.append("High level of background noise detected")
        score = 0.6
    return score, anomalies

def cross_modality_consistency_check(results: Dict[str, DeepfakeAnalysisResult]) -> Tuple[float, List[str]]:
    """
    Check for consistency across modalities (e.g., audio and text analysis).
    """
    score = 0.8
    anomalies = []
    if results.get("audio_analysis") and results.get("text_analysis"):
         if results["audio_analysis"].label != results["text_analysis"].label:
             anomalies.append("Mismatch between audio and text analysis results")
             score = 0.6
    return score, anomalies

In [10]:
def parse_model_output(output: str) -> Tuple[float, List[str]]:
    debug_print(f"Parsing model output: {output}")
    score_match = re.search(r"Score:?\s*(0\.\d+|1\.0)", output, re.IGNORECASE)
    score = float(score_match.group(1)) if score_match else 0.5
    anomalies = []
    anomalies_pattern = r"Anomalies:?\s*(.*?)(?:\n|$)"
    anomalies_match = re.search(anomalies_pattern, output, re.IGNORECASE | re.DOTALL)
    if anomalies_match:
        anomalies_text = anomalies_match.group(1)
        if anomalies_text:
            anomalies_text = anomalies_text.strip("[]")
            anomalies = [a.strip().strip('"\'') for a in anomalies_text.split(',') if a.strip()]
    debug_print(f"Parsed output: score={score}, anomalies={anomalies}")
    return score, anomalies

async def aggregate_llm_outputs(prompt: str, content: str, modality: str="generic") -> Tuple[float, List[str], Dict[str, float]]:
    debug_print(f"Aggregating LLM outputs for modality {modality}")
    scores = []
    all_anomalies = []
    model_scores = {}

    def process_model(model, model_name):
        try:
            response = model(f"{prompt}\n\nContent: {content}")
            s, a = parse_model_output(response)
            scores.append(s)
            all_anomalies.extend(a)
            model_scores[model_name] = s
            debug_print(f"{model_name} score: {s}, anomalies: {a}")
        except Exception as e:
            debug_print(f"Error with {model_name}: {e}")

    with ThreadPoolExecutor() as executor:
        futures = []
        for i, model in enumerate(models[modality]["vllm"]):
            model_name = f"vllm_{modality}_{i}"
            futures.append(executor.submit(process_model, model, model_name))
        for i, model in enumerate(models[modality]["groq"]):
            model_name = f"groq_{modality}_{i}"
            futures.append(executor.submit(process_model, model, model_name))
        for future in as_completed(futures):
            future.result()

    agg_score = float(np.mean(scores)) if scores else 0.5
    unique_anomalies = list(set(all_anomalies))
    debug_print(f"Aggregated score for {modality}: {agg_score}, anomalies: {unique_anomalies}")
    return agg_score, unique_anomalies, model_scores

In [11]:
# --- AUDIO PROCESSING & ANALYSIS ---
def audio_preprocessing(audio_path: str) -> Tuple[np.ndarray, int]:
    debug_print(f"Loading and preprocessing audio from {audio_path} ...")
    try:
        audio_data, sr = librosa.load(audio_path, sr=16000)
        audio_data = audio_data / np.max(np.abs(audio_data))
        if len(audio_data) > 0:
            b, a = librosa.filters.butter(4, 100/(sr/2), btype='highpass')
            audio_data = librosa.filters.filtfilt(b, a, audio_data)
        debug_print(f"Audio loaded and preprocessed. Sample rate: {sr}, Duration: {len(audio_data)/sr:.2f}s")
        return audio_data, sr
    except Exception as e:
        debug_print(f"Error in audio preprocessing: {e}")
        raise

def extract_audio_features(audio_data: np.ndarray, sr: int) -> Dict[str, Any]:
    debug_print("Extracting audio features...")
    features = {}
    stft = np.abs(librosa.stft(audio_data))
    features["stft"] = stft
    mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sr)
    features["mel_spectrogram"] = mel_spec
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
    features["mfccs"] = mfccs
    spectral_contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sr)
    features["spectral_contrast"] = spectral_contrast
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sr)
    features["chroma"] = chroma
    zcr = librosa.feature.zero_crossing_rate(audio_data)
    features["zero_crossing_rate"] = zcr
    onset_env = librosa.onset.onset_strength(y=audio_data, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    features["tempo"] = tempo[0]
    debug_print(f"Audio features extracted: {list(features.keys())}")
    return features

def detect_audio_anomalies(features: Dict[str, Any]) -> List[str]:
    debug_print("Detecting audio anomalies from features...")
    anomalies = []
    mel_spec = features["mel_spectrogram"]
    if np.std(mel_spec) < 0.1 or np.std(mel_spec) > 10:
        anomalies.append("unusual spectral distribution")
    mfccs = features["mfccs"]
    if np.max(np.diff(mfccs, axis=1)) > 5:
        anomalies.append("abrupt MFCC transitions")
    tempo = features["tempo"]
    if tempo < 40 or tempo > 240:
        anomalies.append("unusual speech tempo")
    zcr = features["zero_crossing_rate"]
    if np.mean(zcr) > 0.3:
        anomalies.append("unusually high zero-crossing rate")
    debug_print(f"Detected audio anomalies: {anomalies}")
    return anomalies

async def advanced_audio_analysis(audio_data: np.ndarray, sr: int, device: torch.device) -> DeepfakeAnalysisResult:
    debug_print("Starting advanced audio analysis...")
    audio_features = extract_audio_features(audio_data, sr)
    signal_anomalies = detect_audio_anomalies(audio_features)
    signal_score = 0.8 if not signal_anomalies else 0.6
    # Additional audio checks for echo and noise
    additional_score, additional_anomalies = additional_audio_checks(audio_data, sr)
    combined_score = (signal_score + additional_score) / 2
    combined_anomalies = list(set(signal_anomalies + additional_anomalies))
    feature_summary = (
        f"Audio duration: {len(audio_data)/sr:.2f}s, Sample rate: {sr}Hz\n"
        f"Mean amplitude: {np.mean(np.abs(audio_data)):.4f}, Max amplitude: {np.max(np.abs(audio_data)):.4f}\n"
        f"Detected tempo: {audio_features['tempo']:.2f} BPM\n"
        f"Mean zero-crossing rate: {np.mean(audio_features['zero_crossing_rate']):.4f}\n"
        f"Signal-to-noise ratio estimate: {np.mean(audio_data**2) / np.std(audio_data):.4f}\n"
        f"Traditional analysis anomalies: {signal_anomalies}\n"
        f"Additional audio checks: {additional_anomalies}"
    )
    prompt = "Based on the audio features and additional checks, provide a deepfake confidence score (0-1) and list specific audio anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="audio")
    final_score = (combined_score + llm_score) / 2
    confidence = 1.0 - np.std([combined_score, llm_score])
    final_anomalies = list(set(combined_anomalies + llm_anomalies))
    return DeepfakeAnalysisResult(
        score=final_score,
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=confidence,
        method="advanced_audio_analysis (signal processing + LLM + additional checks)",
        anomalies=final_anomalies,
        explanation="Advanced audio analysis combining traditional feature extraction, additional echo/noise checks, and LLM insights.",
        model_scores=model_scores
    )

In [12]:
# --- VIDEO PROCESSING & ANALYSIS ---
def video_preprocessing(video_path: str) -> Dict[str, Any]:
    debug_print(f"Loading video from {video_path} ...")
    try:
        video = VideoFileClip(video_path)
        frame_count = int(video.fps * video.duration)
        sample_rate = max(1, frame_count // 30)
        frames = [video.get_frame(i / video.fps) for i in range(0, frame_count, sample_rate)]
        audio = video.audio.to_soundarray() if video.audio else None
        metadata = {
            "duration": video.duration,
            "fps": video.fps,
            "frame_count": frame_count,
            "resolution": f"{video.size[0]}x{video.size[1]}",
            "file_size": os.path.getsize(video_path),
            "file_path": video_path
        }
        debug_print(f"Video preprocessed: {len(frames)} frames extracted, Audio present: {'Yes' if audio is not None else 'No'}")
        return {
            "frames": frames,
            "audio": audio,
            "metadata": metadata,
            "image": frames[0] if frames else None,
            "text": "Extracted text placeholder"
        }
    except Exception as e:
        debug_print(f"Error in video preprocessing: {e}")
        raise

def extract_video_features(frames: List[np.ndarray]) -> Dict[str, Any]:
    debug_print(f"Extracting video features from {len(frames)} frames...")
    features = {}
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
    face_landmarks_sequence = []
    face_embeddings = []
    optical_flow_metrics = []
    frame_diffs = []
    blur_metrics = []
    compression_metrics = []
    for i in range(len(frames)):
        frame_rgb = cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB) if frames[i].shape[2] == 3 else frames[i]
        face_landmarks = face_mesh.process(frame_rgb).multi_face_landmarks
        if face_landmarks:
            face_landmarks_sequence.append(face_landmarks[0].landmark)
            face_locations = face_recognition.face_locations(frame_rgb)
            if face_locations:
                face_encoding = face_recognition.face_encodings(frame_rgb, face_locations)[0]
                face_embeddings.append(face_encoding)
        if i > 0:
            prev_gray = cv2.cvtColor(frames[i-1], cv2.COLOR_BGR2GRAY)
            curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
            diff = cv2.absdiff(prev_gray, curr_gray)
            frame_diffs.append(np.mean(diff))
            flow = np.mean(diff)
            optical_flow_metrics.append(flow)
        gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        blur_metric = cv2.Laplacian(gray, cv2.CV_64F).var()
        blur_metrics.append(blur_metric)
        compression_metric = np.mean(cv2.Canny(gray, 100, 200))
        compression_metrics.append(compression_metric)
    features["face_landmarks_sequence"] = face_landmarks_sequence
    features["face_embeddings"] = face_embeddings
    features["optical_flow_metrics"] = optical_flow_metrics
    features["frame_diffs"] = frame_diffs
    features["blur_metrics"] = blur_metrics
    features["compression_metrics"] = compression_metrics
    debug_print(f"Video features extracted: {list(features.keys())}")
    debug_print(f"Found facial landmarks in {len(face_landmarks_sequence)}/{len(frames)} frames")
    face_mesh.close()
    return features

def detect_video_anomalies(features: Dict[str, Any]) -> List[str]:
    debug_print("Detecting video anomalies from features...")
    anomalies = []
    face_landmarks = features.get("face_landmarks_sequence", [])
    if len(face_landmarks) > 1:
        landmark_diffs = []
        for i in range(1, len(face_landmarks)):
            diff = sum(
                np.sqrt((face_landmarks[i][j].x - face_landmarks[i-1][j].x)**2 +
                        (face_landmarks[i][j].y - face_landmarks[i-1][j].y)**2)
                for j in range(min(5, len(face_landmarks[i])))
            )
            landmark_diffs.append(diff)
        if np.max(landmark_diffs) > 0.5:
            anomalies.append("abrupt facial landmark movements")
    face_embeddings = features.get("face_embeddings", [])
    if len(face_embeddings) > 1:
        embedding_diffs = []
        for i in range(1, len(face_embeddings)):
            diff = cosine(face_embeddings[i], face_embeddings[i-1])
            embedding_diffs.append(diff)
        if np.max(embedding_diffs) > 0.3:
            anomalies.append("inconsistent face identity")
    frame_diffs = features.get("frame_diffs", [])
    if frame_diffs:
        if np.std(frame_diffs) < 0.01:
            anomalies.append("unnaturally smooth motion")
        if np.max(frame_diffs) / (np.mean(frame_diffs) + 1e-6) > 10:
            anomalies.append("erratic frame changes")
    blur_metrics = features.get("blur_metrics", [])
    if blur_metrics:
        if np.std(blur_metrics) / (np.mean(blur_metrics) + 1e-6) > 0.8:
            anomalies.append("inconsistent blur levels")
    compression_metrics = features.get("compression_metrics", [])
    if compression_metrics:
        if np.std(compression_metrics) / (np.mean(compression_metrics) + 1e-6) > 0.5:
            anomalies.append("inconsistent compression artifacts")
    debug_print(f"Detected video anomalies: {anomalies}")
    return anomalies

async def advanced_video_analysis(video_data: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    debug_print("Starting advanced video analysis...")
    frames = video_data.get("frames", [])
    if not frames:
        debug_print("No frames available for analysis")
        return DeepfakeAnalysisResult(
            score=0.5,
            label="INCONCLUSIVE",
            confidence=0.0,
            method="advanced_video_analysis",
            anomalies=["no frames available for analysis"],
            explanation="Could not perform video analysis due to lack of frame data."
        )
    video_features = extract_video_features(frames)
    cv_anomalies = detect_video_anomalies(video_features)
    # Additional background consistency check
    bg_score, bg_anomalies = check_background_consistency(frames)
    cv_score = 0.8 if not cv_anomalies else 0.6
    # Combine scores for video analysis (simple average)
    combined_video_score = (cv_score + bg_score) / 2
    combined_anomalies = list(set(cv_anomalies + bg_anomalies))
    feature_summary = (
        f"Video features: {len(frames)} frames, Resolution: {video_data['metadata']['resolution']}\n"
        f"Facial landmarks detected in {len(video_features.get('face_landmarks_sequence', []))}/{len(frames)} frames\n"
        f"Mean frame difference: {np.mean(video_features.get('frame_diffs', [0])):.4f}\n"
        f"Blur consistency: {np.std(video_features.get('blur_metrics', [0]))/(np.mean(video_features.get('blur_metrics', [1]))+1e-6):.4f}\n"
        f"Traditional analysis anomalies: {cv_anomalies}\n"
        f"Background consistency anomalies: {bg_anomalies}"
    )
    prompt = "Based on the video features, background consistency, and additional checks, provide a deepfake confidence score (0-1) and list specific video anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="video")
    final_score = (combined_video_score + llm_score) / 2
    confidence = 1.0 - np.std([combined_video_score, llm_score])
    final_anomalies = list(set(combined_anomalies + llm_anomalies))
    return DeepfakeAnalysisResult(
        score=final_score,
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=confidence,
        method="advanced_video_analysis (computer vision + background consistency + LLM)",
        anomalies=final_anomalies,
        explanation="Advanced video analysis combining computer vision features, background consistency checks, and multiagent LLM insights.",
        model_scores=model_scores
    )

In [13]:
# --- IMAGE PROCESSING & ANALYSIS ---
def image_preprocessing(image_path: str) -> np.ndarray:
    debug_print(f"Loading and preprocessing image from {image_path} ...")
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image from {image_path}")
        max_dim = 1024
        h, w = image.shape[:2]
        if max(h, w) > max_dim:
            scale = max_dim / max(h, w)
            image = cv2.resize(image, (int(w * scale), int(h * scale)))
            debug_print(f"Image resized from {w}x{h} to {int(w * scale)}x{int(h * scale)}")
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        debug_print(f"Image loaded and preprocessed. Shape: {image_rgb.shape}")
        return image_rgb
    except Exception as e:
        debug_print(f"Error in image preprocessing: {e}")
        raise

def extract_image_features(image: np.ndarray) -> Dict[str, Any]:
    debug_print("Extracting image features...")
    features = {}
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    features["face_count"] = len(faces)
    if len(faces) > 0:
        x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])
        face_roi = gray[y:y+h, x:x+w]
        features["face_roi"] = face_roi
        face_landmarks = face_recognition.face_landmarks(image)
        features["face_landmarks"] = face_landmarks
        face_encodings = face_recognition.face_encodings(image)
        if face_encodings:
            features["face_encoding"] = face_encodings[0]
    temp_path = "temp_ela.jpg"
    cv2.imwrite(temp_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, 90])
    low_qual = cv2.imread(temp_path)
    low_qual_rgb = cv2.cvtColor(low_qual, cv2.COLOR_BGR2RGB)
    ela = cv2.absdiff(image, low_qual_rgb)
    features["ela"] = cv2.cvtColor(ela, cv2.COLOR_RGB2GRAY)
    if os.path.exists(temp_path):
        os.remove(temp_path)
    noise = cv2.medianBlur(gray, 5) - gray
    features["noise_pattern"] = noise
    edges = cv2.Canny(gray, 100, 200)
    features["edges"] = edges
    dct_blocks = np.zeros_like(gray, dtype=float)
    block_size = 8
    for i in range(0, gray.shape[0] - block_size, block_size):
        for j in range(0, gray.shape[1] - block_size, block_size):
            block = gray[i:i+block_size, j:j+block_size].astype(float)
            dct_block = cv2.dct(block)
            dct_blocks[i:i+block_size, j:j+block_size] = dct_block
    features["dct_blocks"] = dct_blocks
    debug_print(f"Image features extracted. Faces found: {features['face_count']}")
    return features

def detect_image_anomalies(features: Dict[str, Any]) -> List[str]:
    debug_print("Detecting image anomalies from features...")
    anomalies = []
    if features["face_count"] > 0:
        face_landmarks = features.get("face_landmarks", [])
        if face_landmarks:
            landmarks = face_landmarks[0]
            left_eye = np.mean(landmarks.get("left_eye", [[0, 0]]), axis=0)
            right_eye = np.mean(landmarks.get("right_eye", [[0, 0]]), axis=0)
            if abs(left_eye[1] - right_eye[1]) > 10:
                anomalies.append("asymmetric eye alignment")
            nose_tip = landmarks.get("nose_tip", [[0, 0]])[0]
            top_lip = np.mean(landmarks.get("top_lip", [[0, 0]]), axis=0)
            if abs((nose_tip[1] - left_eye[1]) - (top_lip[1] - nose_tip[1])) > 15:
                anomalies.append("unusual facial proportions")
        ela = features.get("ela")
        if ela is not None:
            ela_mean = np.mean(ela)
            ela_std = np.std(ela)
            if ela_mean > 10 or ela_std > 15:
                anomalies.append("potential image manipulation (ELA)")
        noise = features.get("noise_pattern")
        if noise is not None:
            noise_mean = np.mean(np.abs(noise))
            noise_std = np.std(noise)
            if noise_std / (noise_mean + 1e-6) > 3 or noise_mean < 0.5:
                anomalies.append("unnatural noise pattern")
        dct_blocks = features.get("dct_blocks")
        if dct_blocks is not None:
            block_diff = cv2.Laplacian(dct_blocks, cv2.CV_64F).var()
            if block_diff > 500:
                anomalies.append("unusual compression artifacts")
    else:
        anomalies.append("no faces detected for analysis")
    debug_print(f"Detected image anomalies: {anomalies}")
    return anomalies

async def advanced_image_analysis(image: np.ndarray, device: torch.device) -> DeepfakeAnalysisResult:
    debug_print("Starting advanced image analysis...")
    image_features = extract_image_features(image)
    cv_anomalies = detect_image_anomalies(image_features)
    cv_score = 0.8 if not cv_anomalies else 0.6
    # Additional face forgery detection step
    forgery_score, forgery_anomalies = detect_face_forgery(image)
    combined_score = (cv_score + forgery_score) / 2
    combined_anomalies = list(set(cv_anomalies + forgery_anomalies))
    feature_summary = (
        f"Image features: Resolution: {image.shape[1]}x{image.shape[0]}, Faces detected: {image_features['face_count']}\n"
        f"ELA mean: {np.mean(image_features.get('ela', np.zeros(1))):.4f}, std: {np.std(image_features.get('ela', np.zeros(1))):.4f}\n"
        f"Noise pattern: mean: {np.mean(np.abs(image_features.get('noise_pattern', np.zeros(1)))):.4f}, std: {np.std(image_features.get('noise_pattern', np.zeros(1))):.4f}\n"
        f"Traditional analysis anomalies: {cv_anomalies}\n"
        f"Face forgery anomalies: {forgery_anomalies}"
    )
    prompt = "Based on the image features, including face forgery detection, provide a deepfake confidence score (0-1) and list specific image anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="image")
    final_score = (combined_score + llm_score) / 2
    confidence = 1.0 - np.std([combined_score, llm_score])
    final_anomalies = list(set(combined_anomalies + llm_anomalies))
    return DeepfakeAnalysisResult(
        score=final_score,
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=confidence,
        method="advanced_image_analysis (computer vision + face forgery detection + LLM)",
        anomalies=final_anomalies,
        explanation="Advanced image analysis combining computer vision features, additional face forgery checks, and multiagent LLM insights.",
        model_scores=model_scores
    )

In [14]:
# --- TEXT ANALYSIS ---
async def analyze_text_content(text: str) -> DeepfakeAnalysisResult:
    debug_print("Starting text content analysis...")
    word_count = len(text.split())
    sent_count = len(re.split(r'[.!?]+', text))
    avg_word_len = sum(len(word) for word in text.split()) / max(1, word_count)
    feature_summary = (
        f"Text statistics: {word_count} words, {sent_count} sentences\n"
        f"Average word length: {avg_word_len:.2f}\n"
        f"Sample text (truncated): {text[:500]}...\n"
    )
    prompt = "Based on the text features, provide a score (0-1) indicating if this text was AI-generated or manipulated and list specific text anomalies."
    llm_score, llm_anomalies, model_scores = await aggregate_llm_outputs(prompt, feature_summary, modality="text")
    return DeepfakeAnalysisResult(
        score=llm_score,
        label="HUMAN" if llm_score > 0.7 else "AI-GENERATED",
        confidence=0.8,
        method="text_analysis (LLM-based)",
        anomalies=llm_anomalies,
        explanation="Text analysis using multiagent LLM insights to detect AI-generated content.",
        model_scores=model_scores
    )

In [15]:
def cross_modality_consistency_check(results: Dict[str, DeepfakeAnalysisResult]) -> Tuple[float, List[str]]:
    score = 0.8
    anomalies = []
    if results.get("audio_analysis") and results.get("text_analysis"):
         if results["audio_analysis"].label != results["text_analysis"].label:
             anomalies.append("Mismatch between audio and text analysis results")
             score = 0.6
    return score, anomalies

In [16]:
def calculate_multimodal_score(results: Dict[str, DeepfakeAnalysisResult]) -> float:
    debug_print("Calculating multimodal score...")
    scores = []
    weights = []
    for modality, result in results.items():
        if result:
            scores.append(result.score)
            weights.append(result.confidence)
    if not scores:
        debug_print("No valid results found for multimodal scoring")
        return 0.5
    total_weight = sum(weights)
    if total_weight == 0:
        weighted_avg = sum(scores) / len(scores)
    else:
        norm_weights = [w / total_weight for w in weights]
        weighted_avg = sum(s * w for s, w in zip(scores, norm_weights))
    debug_print(f"Multimodal weighted score: {weighted_avg:.4f}")
    return weighted_avg

def generate_verdict(score: float, results: Dict[str, DeepfakeAnalysisResult]) -> str:
    if score > 0.8:
        return "AUTHENTIC - High confidence that this content is genuine"
    elif score > 0.6:
        return "LIKELY AUTHENTIC - Moderate confidence that this content is genuine"
    elif score > 0.4:
        return "INCONCLUSIVE - Analysis unable to determine authenticity"
    elif score > 0.2:
        return "LIKELY MANIPULATED - Moderate confidence that this content contains manipulations"
    else:
        return "MANIPULATED - High confidence that this content is manipulated or synthetic"

def collect_evidence(results: Dict[str, DeepfakeAnalysisResult]) -> List[Evidence]:
    evidence_list = []
    for modality, result in results.items():
        if result:
            for anomaly in result.anomalies:
                evidence_list.append(
                    Evidence(
                        type=modality,
                        description=anomaly,
                        confidence=result.confidence,
                        method=result.method
                    )
                )
    return evidence_list

def generate_recommendations(report: MultimodalAnalysisReport) -> List[str]:
    recommendations = []
    if report.multimodal_score < 0.4:
        recommendations.append("This content should be treated as potentially manipulated and not be used as evidence without further forensic analysis.")
    if report.video_analysis and report.video_analysis.score < 0.5:
        recommendations.append("Video content shows significant signs of manipulation and should be verified through alternative sources.")
    if report.audio_analysis and report.audio_analysis.score < 0.5:
        recommendations.append("Audio content appears to be synthetic or manipulated. Verify the source and content through other means.")
    if report.image_analysis and report.image_analysis.score < 0.5:
        recommendations.append("Image analysis indicates potential manipulation. Consider requesting the original, unprocessed image file.")
    if report.text_analysis and report.text_analysis.score < 0.5:
        recommendations.append("Text content appears to be AI-generated. Verify authorship through additional means.")
    recommendations.append("For definitive analysis, consider submitting this content to a professional forensic lab specialized in digital media authentication.")
    return recommendations

In [17]:
async def enhance_report_with_mcp(report: MultimodalAnalysisReport) -> str:
    config = {
        "mcpServers": {
            "http": {
                "url": "http://localhost:8931/sse"  # Adjust this according to your MCP server settings.
            },
            "playwright": {
                "command": "npx",
                "args": ["@playwright/mcp@latest"],
                "env": {
                    "DISPLAY": ":1"
                }
            }
        }
    }
    client = MCPClient.from_dict(config)
    llm = VLLM(
        model="meta-llama/Llama-3-70b-hf",
        trust_remote_code=True,
        max_new_tokens=128,
        top_k=10,
        top_p=0.95,
        temperature=0.8,
    )
    agent = MCPAgent(llm=llm, client=client, max_steps=30)
    query = ("Using Google search, please find extra contextual information about recent deepfake detection trends "
             "and forensic analysis in digital media. Provide a concise summary with relevant links.")
    result = await agent.run(query, max_steps=30)
    report.metadata["mcp_enhancement"] = result
    if client.sessions:
        await client.close_all_sessions()
    return result

In [18]:
#######################################
###   MAIN PIPELINE ORCHESTRATION  ###
#######################################
class DeepfakePipeline:
    def __init__(self):
        debug_print("Initializing DeepfakePipeline...")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        debug_print(f"Using device: {self.device}")

    async def analyze_file(self, file_path: str) -> MultimodalAnalysisReport:
        debug_print(f"Starting analysis for file: {file_path}")
        start_time = datetime.now()
        file_ext = os.path.splitext(file_path)[1].lower()
        file_info = {
            "path": file_path,
            "size": os.path.getsize(file_path),
            "type": "unknown",
            "extension": file_ext
        }
        results = {
            "video_analysis": None,
            "audio_analysis": None,
            "image_analysis": None,
            "text_analysis": None
        }
        try:
            if file_ext in ['.mp4', '.mov', '.avi', '.mkv', '.webm']:
                file_info["type"] = "video"
                video_data = video_preprocessing(file_path)
                tasks = []
                tasks.append(advanced_video_analysis(video_data, self.device))
                if video_data.get("audio") is not None:
                    audio_data, sr = video_data["audio"], 44100
                    tasks.append(advanced_audio_analysis(audio_data, sr, self.device))
                if video_data.get("frames"):
                    image = video_data["frames"][0]
                    tasks.append(advanced_image_analysis(image, self.device))
                if video_data.get("text"):
                    tasks.append(analyze_text_content(video_data["text"]))
                completed_tasks = await asyncio.gather(*tasks)
                results["video_analysis"] = completed_tasks[0]
                task_idx = 1
                if video_data.get("audio") is not None:
                    results["audio_analysis"] = completed_tasks[task_idx]
                    task_idx += 1
                if video_data.get("frames"):
                    results["image_analysis"] = completed_tasks[task_idx]
                    task_idx += 1
                if video_data.get("text"):
                    results["text_analysis"] = completed_tasks[task_idx]
            elif file_ext in ['.wav', '.mp3', '.ogg', '.flac']:
                file_info["type"] = "audio"
                audio_data, sr = audio_preprocessing(file_path)
                results["audio_analysis"] = await advanced_audio_analysis(audio_data, sr, self.device)
                text_content = "Audio transcript would be here in a real implementation."
                results["text_analysis"] = await analyze_text_content(text_content)
            elif file_ext in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
                file_info["type"] = "image"
                image = image_preprocessing(file_path)
                results["image_analysis"] = await advanced_image_analysis(image, self.device)
            elif file_ext in ['.txt', '.md', '.rtf', '.doc', '.docx', '.pdf']:
                file_info["type"] = "text"
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text_content = f.read()
                results["text_analysis"] = await analyze_text_content(text_content)
            else:
                debug_print(f"Unsupported file type: {file_ext}")
                raise ValueError(f"Unsupported file type: {file_ext}")
            multimodal_score = calculate_multimodal_score(results)
            # Cross modality consistency check
            cross_score, cross_anomalies = cross_modality_consistency_check(results)
            multimodal_score = (multimodal_score + cross_score) / 2
            verdict = generate_verdict(multimodal_score, results)
            evidence = collect_evidence(results)
            processing_time = (datetime.now() - start_time).total_seconds()
            confidence_matrix = {
                modality: {
                    "score": result.score if result else None,
                    "confidence": result.confidence if result else None
                }
                for modality, result in results.items()
            }
            report = MultimodalAnalysisReport(
                case_id=f"case_{datetime.now().strftime('%Y%m%d%H%M%S')}",
                file_info=file_info,
                video_analysis=results["video_analysis"],
                audio_analysis=results["audio_analysis"],
                image_analysis=results["image_analysis"],
                text_analysis=results["text_analysis"],
                multimodal_score=multimodal_score,
                verdict=verdict,
                evidence=evidence,
                metadata={
                    "pipeline_version": "1.0.0",
                    "analysis_timestamp": datetime.now().isoformat(),
                    "device": str(self.device),
                    "cross_modality_consistency": {
                        "score": cross_score,
                        "anomalies": cross_anomalies
                    }
                },
                confidence_matrix=confidence_matrix,
                processing_time=processing_time
            )
            report.recommendations = generate_recommendations(report)
            # Enhance report with additional online context using MCP.
            mcp_context = await enhance_report_with_mcp(report)
            debug_print(f"MCP enhancement result added to report metadata: {mcp_context}")
            return report
        except Exception as e:
            debug_print(f"Error in analyze_file: {e}")
            raise

In [19]:
class PipelineState(BaseModel):
    file_path: str
    report: Optional[MultimodalAnalysisReport] = None
    current_step: str = "initialize"
    error: Optional[str] = None
    results: Dict[str, Any] = Field(default_factory=dict)

async def initialize_analysis(state: PipelineState) -> PipelineState:
    debug_print(f"Initializing analysis for {state.file_path}")
    try:
        state.current_step = "processing"
        return state
    except Exception as e:
        state.error = str(e)
        state.current_step = "error"
        return state

async def process_file(state: PipelineState) -> PipelineState:
    try:
        pipeline = DeepfakePipeline()
        state.report = await pipeline.analyze_file(state.file_path)
        state.current_step = "complete"
        return state
    except Exception as e:
        state.error = str(e)
        state.current_step = "error"
        return state

def create_pipeline_graph() -> StateGraph:
    workflow = StateGraph(PipelineState)
    workflow.add_node("initialize", initialize_analysis)
    workflow.add_node("processing", process_file)
    workflow.add_edge("initialize", "processing")
    workflow.add_edge("processing", END)
    workflow.add_conditional_edges("initialize", lambda state: "error" if state.error else "processing")
    workflow.add_conditional_edges("processing", lambda state: "error" if state.error else END)
    return workflow.compile()

In [20]:
async def main():
    import argparse
    parser = argparse.ArgumentParser(description="Deepfake Detection Pipeline with MCP Enhancement using Groq and vLLM")
    parser.add_argument('file_path', help='Path to the file to analyze')
    parser.add_argument('--debug', action='store_true', help='Enable debug output')
    parser.add_argument('--output', help='Path to save the JSON report')
    args = parser.parse_args()
    global DEBUG
    DEBUG = args.debug
    debug_print(f"Starting deepfake analysis for file: {args.file_path}")
    try:
        workflow = create_pipeline_graph()
        state = PipelineState(file_path=args.file_path)
        final_state = await workflow.ainvoke(state)
        if final_state.error:
            print(f"Error: {final_state.error}")
            return 1
        report = final_state.report
        print("\n" + "="*50)
        print(f"DEEPFAKE ANALYSIS REPORT - {report.case_id}")
        print("="*50)
        print(f"File: {os.path.basename(args.file_path)} ({report.file_info['type']})")
        print(f"Overall Score: {report.multimodal_score:.2f} (0=Fake, 1=Real)")
        print(f"Verdict: {report.verdict}")
        print("-"*50)
        for modality in ["video", "audio", "image", "text"]:
            result = getattr(report, f"{modality}_analysis")
            if result:
                print(f"{modality.upper()} Analysis: {result.score:.2f} ({result.label})")
                if result.anomalies:
                    print(f"  Detected anomalies: {', '.join(result.anomalies)}")
        print("-"*50)
        print("RECOMMENDATIONS:")
        for i, rec in enumerate(report.recommendations, 1):
            print(f"{i}. {rec}")
        if args.output:
            with open(args.output, 'w') as f:
                f.write(report.json(indent=2))
            print(f"\nDetailed report saved to {args.output}")
        return 0
    except Exception as e:
        print(f"Error in main execution: {e}")
        import traceback
        traceback.print_exc()
        return 1

In [21]:
async def run_deepfake_enhanced_ui():
    from google.colab import files
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    import json, time, matplotlib.pyplot as plt
    from matplotlib.colors import LinearSegmentedColormap
    import numpy as np, io, base64
    from datetime import datetime

    colors = [(0.8, 0.2, 0.2), (1, 0.6, 0), (0.2, 0.8, 0.2)]
    cmap = LinearSegmentedColormap.from_list("deepfake_cmap", colors, N=100)
    dashboard_html = """
    Deepfake Detection Pipeline with MCP Enhancement using Groq and vLLM
    Upload media files to analyze for potential manipulations across multiple modalities.
    """
    display(HTML(dashboard_html))
    tabs = widgets.Tab()
    tab_analysis = widgets.VBox()
    tab_results = widgets.VBox()
    tab_logs = widgets.VBox()
    tabs.children = [tab_analysis, tab_results, tab_logs]
    tabs.set_title(0, "Analysis")
    tabs.set_title(1, "Results")
    tabs.set_title(2, "Debug Logs")
    upload_button = widgets.Button(
        description='Upload File',
        disabled=False,
        button_style='primary',
        tooltip='Click to upload a file for analysis',
        icon='upload',
        layout=widgets.Layout(width='200px')
    )
    debug_checkbox = widgets.Checkbox(value=True, description='Enable debug output', disabled=False)
    file_info = widgets.HTML(value="No file selected", layout=widgets.Layout(margin="10px 0px"))
    status_label = widgets.HTML(value="Status: Ready", layout=widgets.Layout(margin="10px 0px"))
    progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Overall:',
                                       bar_style='info', style={'bar_color': '#007bff'},
                                       orientation='horizontal', layout=widgets.Layout(width='100%', margin="10px 0px"))
    step_progress = widgets.HTML(value="Waiting for file upload...", layout=widgets.Layout(margin="10px 0px"))
    log_output = widgets.Output(layout=widgets.Layout(height='300px', border='1px solid #ddd', overflow_y='auto',
                                                      padding='10px', margin="10px 0px"))
    result_output = widgets.Output(layout=widgets.Layout(height='500px', border='1px solid #ddd', overflow_y='auto',
                                                         padding='10px', margin="10px 0px"))
    visualization_area = widgets.Output(layout=widgets.Layout(height='auto', padding='10px', margin="20px 0px"))
    export_options = widgets.HBox([
        widgets.Button(description='Download JSON', icon='download', button_style='success', layout=widgets.Layout(width='150px')),
        widgets.Button(description='Download PDF', icon='file-pdf', button_style='info', layout=widgets.Layout(width='150px')),
        widgets.Button(description='View Full Report', icon='eye', button_style='warning', layout=widgets.Layout(width='150px'))
    ], layout=widgets.Layout(display='none'))
    tab_analysis.children = [widgets.HBox([upload_button, debug_checkbox]), file_info, status_label,
                             progress_bar, step_progress, log_output]
    tab_results.children = [visualization_area, result_output, export_options]
    debug_controls = widgets.HBox([widgets.Button(description='Clear Logs', icon='trash', button_style='danger',
                                                  layout=widgets.Layout(width='120px')), widgets.Checkbox(value=True, description='Auto-scroll', disabled=False)])
    debug_output = widgets.Output(layout=widgets.Layout(height='400px', border='1px solid #ddd', overflow_y='auto',
                                                       padding='10px', font_family='monospace', margin="10px 0px"))
    tab_logs.children = [debug_controls, debug_output]
    display(tabs)
    current_file_path = None
    process_stages = {
        "preprocessing": {"started": False, "completed": False, "weight": 10},
        "feature_extraction": {"started": False, "completed": False, "weight": 20},
        "model_processing": {"started": False, "completed": False, "weight": 40},
        "analysis": {"started": False, "completed": False, "weight": 20},
        "report_generation": {"started": False, "completed": False, "weight": 10}
    }
    def update_progress(stage, status, message=None, substage=None):
        if stage in process_stages:
            if status == "start":
                process_stages[stage]["started"] = True
            elif status == "complete":
                process_stages[stage]["started"] = True
                process_stages[stage]["completed"] = True
        total_weight = sum(stage_info["weight"] for stage_info in process_stages.values())
        completed_weight = sum(stage_info["weight"] for stage_info in process_stages.values() if stage_info["completed"])
        in_progress_weight = sum(stage_info["weight"] * 0.5 for stage_info in process_stages.values() if stage_info["started"] and not stage_info["completed"])
        progress_value = int((completed_weight + in_progress_weight) / total_weight * 100)
        progress_bar.value = progress_value
        stage_display = stage.replace("_", " ").title()
        if status == "start":
            status_label.value = f"Status: Processing - {stage_display}"
            step_progress.value = f"Current step: {stage_display}{' - ' + substage if substage else ''}"
        elif status == "complete":
            step_progress.value = f"Completed: {stage_display}{' - ' + substage if substage else ''}"
        elif status == "error":
            status_label.value = f"Status: Error in {stage_display}"
            step_progress.value = f"Error: {message}"
        timestamp = datetime.now().strftime("%H:%M:%S")
        with debug_output:
            print(f"[{timestamp}] {stage_display}: {status.upper()} {message if message else ''}")
    def ui_debug_print(msg: str):
        timestamp = datetime.now().strftime("%H:%M:%S")
        with debug_output:
            print(f"[{timestamp}] [DEBUG] {msg}")
    global debug_print
    debug_print = ui_debug_print
    def visualize_results(report):
        with visualization_area:
            clear_output()
            fig = plt.figure(figsize=(10, 8))
            ax_overall = plt.subplot2grid((3, 3), (0, 0), colspan=3)
            overall_score = report.multimodal_score
            overall_color = cmap(overall_score)
            ax_overall.barh(['Overall'], [overall_score], color=overall_color, height=0.5)
            ax_overall.set_xlim(0, 1)
            ax_overall.set_title(f'Overall Analysis: {report.verdict}', fontsize=14)
            ax_overall.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
            ax_modalities = plt.subplot2grid((3, 3), (1, 0), colspan=3)
            modalities = []
            scores = []
            colors = []
            for modality in ["video", "audio", "image", "text"]:
                result = getattr(report, f"{modality}_analysis")
                if result:
                    modalities.append(modality.capitalize())
                    scores.append(result.score)
                    colors.append(cmap(result.score))
            ax_modalities.barh(modalities, scores, color=colors, height=0.5)
            ax_modalities.set_xlim(0, 1)
            ax_modalities.set_title('Analysis by Modality', fontsize=12)
            ax_modalities.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
            for i, score in enumerate(scores):
                ax_modalities.text(min(max(score + 0.05, 0.1), 0.95), i, f"{score:.2f}",
                                   va='center', ha='center', fontweight='bold')
            ax_anomalies = plt.subplot2grid((3, 3), (2, 0), colspan=3, rowspan=1)
            ax_anomalies.axis('off')
            anomaly_text = "Key Anomalies Detected:\n"
            anomaly_count = 0
            for modality in ["video", "audio", "image", "text"]:
                result = getattr(report, f"{modality}_analysis")
                if result and result.anomalies:
                    anomaly_text += f"\n{modality.capitalize()}: {', '.join(result.anomalies[:3])}"
                    if len(result.anomalies) > 3:
                        anomaly_text += f" (+ {len(result.anomalies) - 3} more)"
                    anomaly_count += len(result.anomalies)
            if anomaly_count == 0:
                anomaly_text += "\nNo significant anomalies detected."
            ax_anomalies.text(0.5, 0.5, anomaly_text,
                              ha='center', va='center', fontsize=10,
                              bbox=dict(boxstyle='round', facecolor='#f8f9fa', alpha=0.8))
            plt.tight_layout()
            plt.show()
    def display_report(report):
        with result_output:
            clear_output()
            display(HTML(f"""
            <h2>Deepfake Analysis Report</h2>
            <b>Case ID:</b> {report.case_id}<br>
            <b>VERDICT:</b> {report.verdict}<br>
            <b>Overall Score:</b> {report.multimodal_score:.2f} (0=Fake, 1=Real)<br>
            <h3>File Information</h3>
            <b>File Type:</b> {report.file_info['type'].upper()}<br>
            <b>File Size:</b> {report.file_info['size'] / (1024*1024):.2f} MB<br>
            <b>Extension:</b> {report.file_info['extension']}<br>
            <h3>Analysis Results</h3>
            """))
            for modality in ["video", "audio", "image", "text"]:
                result = getattr(report, f"{modality}_analysis")
                if result:
                    score = result.score
                    color = '#d4edda' if score > 0.7 else '#f8d7da' if score < 0.3 else '#fff3cd'
                    display(HTML(f"""
                    <h4>{modality.upper()} Analysis: {score:.2f} ({result.label})</h4>
                    <b>Confidence:</b> {result.confidence:.2f}<br>
                    <b>Method:</b> {result.method}<br>
                    <b>Detected Anomalies:</b> {', '.join(result.anomalies) if result.anomalies else "No anomalies detected"}<br>
                    """))
            display(HTML(f"""
            <h3>Evidence Summary</h3>
            Total evidence items: {len(report.evidence)}<br>
            <h3>Recommendations</h3>
            {"<br>".join(report.recommendations)}<br>
            <b>Processing time:</b> {report.processing_time:.2f} seconds<br>
            <b>Report generated:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            """))
    def generate_pdf_report(report):
        try:
            return f"deepfake_report_{int(time.time())}.pdf"
        except Exception as e:
            with log_output:
                print(f"Error generating PDF: {e}")
            return None
    def download_json_report(report, b):
        try:
            report_json = report.json(indent=2)
            report_filename = f"deepfake_report_{int(time.time())}.json"
            with open(report_filename, 'w') as f:
                f.write(report_json)
            files.download(report_filename)
        except Exception as e:
            with log_output:
                print(f"Error downloading JSON report: {e}")
    def download_pdf_report(report, b):
        try:
            with log_output:
                print("PDF export functionality is a placeholder - would be implemented with a PDF library")
        except Exception as e:
            with log_output:
                print(f"Error downloading PDF report: {e}")
    def show_full_report(report, b):
        with result_output:
            display(HTML(f"""
            <h3>Full Technical Report</h3>
            <pre>{json.dumps(json.loads(report.json()), indent=2)}</pre>
            """))
    def on_upload_clicked(b):
        with log_output:
            clear_output()
            print("Please select a file to upload...")
        global DEBUG, current_file_path
        DEBUG = debug_checkbox.value
        for stage in process_stages:
            process_stages[stage]["started"] = False
            process_stages[stage]["completed"] = False
        progress_bar.value = 0
        try:
            status_label.value = "Status: Uploading File"
            export_options.layout.display = 'none'
            uploaded = files.upload()
            if not uploaded:
                with log_output:
                    print("No file was uploaded.")
                status_label.value = "Status: Ready"
                step_progress.value = "Waiting for file upload..."
                return
            file_path = list(uploaded.keys())[0]
            file_size_mb = uploaded[file_path].size / (1024*1024)
            current_file_path = file_path
            file_info.value = f"""
            <b>File:</b> {file_path}<br>
            <b>Size:</b> {file_size_mb:.2f} MB<br>
            <b>Type:</b> {file_path.split('.')[-1].upper()}
            """
            with log_output:
                print(f"File uploaded: {file_path} ({file_size_mb:.2f} MB)")
                print("Preparing for analysis...")
            temp_path = f"/tmp/{file_path}"
            with open(temp_path, "wb") as f:
                f.write(uploaded[file_path])
            update_progress("preprocessing", "start", f"Initializing for {file_path}")
            workflow = create_pipeline_graph()
            state = PipelineState(file_path=temp_path)
            with log_output:
                print("Starting analysis workflow...")
                print("This process may take several minutes depending on file size and complexity.")
            asyncio.create_task(run_analysis(workflow, state))
        except Exception as e:
            import traceback
            with log_output:
                print(f"Error starting analysis: {e}")
                traceback.print_exc()
            status_label.value = f"Status: Error - {str(e)[:50]}..."
            step_progress.value = "Analysis failed"
    async def run_analysis(workflow, state):
        global extract_video_features, extract_audio_features, extract_image_features
        try:
            await asyncio.sleep(1)
            update_progress("preprocessing", "complete")
            update_progress("feature_extraction", "start")
            original_extract_video_features = extract_video_features
            original_extract_audio_features = extract_audio_features
            original_extract_image_features = extract_image_features
            def wrapped_extract_video_features(frames):
                update_progress("feature_extraction", "start", substage="Video Features")
                result = original_extract_video_features(frames)
                update_progress("feature_extraction", "complete", substage="Video Features")
                return result
            def wrapped_extract_audio_features(audio_data, sr):
                update_progress("feature_extraction", "start", substage="Audio Features")
                result = original_extract_audio_features(audio_data, sr)
                update_progress("feature_extraction", "complete", substage="Audio Features")
                return result
            def wrapped_extract_image_features(image):
                update_progress("feature_extraction", "start", substage="Image Features")
                result = original_extract_image_features(image)
                update_progress("feature_extraction", "complete", substage="Image Features")
                return result
            extract_video_features = wrapped_extract_video_features
            extract_audio_features = wrapped_extract_audio_features
            extract_image_features = wrapped_extract_image_features
            update_progress("model_processing", "start")
            final_state = await workflow.ainvoke(state)
            extract_video_features = original_extract_video_features
            extract_audio_features = original_extract_audio_features
            extract_image_features = original_extract_image_features
            if final_state.error:
                update_progress("model_processing", "error", final_state.error)
                with log_output:
                    print(f"Error in analysis: {final_state.error}")
                status_label.value = "Status: Analysis Failed"
                return
            update_progress("model_processing", "complete")
            update_progress("analysis", "start")
            await asyncio.sleep(1)
            update_progress("analysis", "complete")
            update_progress("report_generation", "start")
            report = final_state.report
            tabs.selected_index = 1
            visualize_results(report)
            display_report(report)
            export_options.layout.display = 'flex'
            export_options.children[0].on_click(lambda b: download_json_report(report, b))
            export_options.children[1].on_click(lambda b: download_pdf_report(report, b))
            export_options.children[2].on_click(lambda b: show_full_report(report, b))
            update_progress("report_generation", "complete")
            status_label.value = "Status: Analysis Complete"
            step_progress.value = "Analysis completed. See Results for details."
            with log_output:
                print("\n" + "="*50)
                print("Analysis completed successfully!")
                print(f"Overall score: {report.multimodal_score:.2f} - Verdict: {report.verdict}")
                print("="*50)
                print("View the Results tab for detailed report and visualizations.")
        except Exception as e:
            import traceback
            with log_output:
                print(f"Error in analysis process: {e}")
                traceback.print_exc()
            status_label.value = "Status: Error"
            step_progress.value = f"Analysis failed: {str(e)[:100]}"
    tab_logs.children[0].children[0].on_click(lambda b: debug_output.clear_output())
    upload_button.on_click(on_upload_clicked)
    with log_output:
        print("🚀 Deepfake Detection Pipeline with MCP Enhancement using Groq and vLLM initialized")
        print("Click 'Upload File' to begin analysis")
    with debug_output:
        print("[INFO] Debug logs will appear here")
        print("[INFO] Pipeline ready for file upload")

# Execute the UI
run_deepfake_enhanced_ui()

<coroutine object run_deepfake_enhanced_ui at 0x17121a20>